# Tutorial 1: The Costs of Closing Failed Banks
## Replication of Kang, Lowery & Wardlaw (Review of Financial Studies, 2015)
---

## Table of Contents

1. [Economic Framework](#1-economic-framework)
   - 1.1 The Research Question
   - 1.2 The Dynamic Discrete Choice Model
   - 1.3 The CCP Approach (Hotz-Miller 1993)
2. [Data Overview](#2-data-overview)
   - 2.1 Data Sources
   - 2.2 Variable Definitions
   - 2.3 Summary Statistics
3. [First Stage: CCP Estimation](#3-first-stage-ccp-estimation)
   - 3.1 Flexible Logit Specification
   - 3.2 Estimation Results
   - 3.3 Verification and Diagnostics
4. [Second Stage: Forward Simulation](#4-second-stage-forward-simulation)
   - 4.1 State Transition Model
   - 4.2 Monte Carlo Integration
   - 4.3 Verification
5. [Third Stage: Structural GMM Estimation](#5-third-stage-structural-gmm-estimation)
   - 5.1 The Moment Condition
   - 5.2 Continuously-Updated GMM
   - 5.3 Grid-Search Refinement Procedure
   - 5.4 Estimation Results
6. [Inference and Results](#6-inference-and-results)
   - 6.1 Standard Error Computation
   - 6.2 Pre-Estimation Variance Adjustments
7. [Economic Interpretation](#7-economic-interpretation)
   - 7.1 Parameter Interpretation
   - 7.2 Net Monetary Cost Function

---

# 1. Economic Framework

## 1.1 The Research Question

**Economic Question**: What are the structural parameters governing bank regulators' decisions to close a bank?

**Object of Interest**: 
- Discount factor ($\beta$) - how much do regulators value future costs?
- Scale parameter ($\sigma$) - how much heterogeneity exists in unobserved closure costs?
- Net monetary cost function parameters ($\beta_{NMC}$) - how do bank characteristics affect operating costs?

**Identification**: The CCP approach (Hotz-Miller 1993) identifies the value function difference from observed closure probabilities. The key identifying assumption is that unobserved shocks follow the Type-I Extreme Value distribution.

**Sample**: 22,269 bank-quarter observations during the S&L Crisis era (1985Q4-1992Q4).

## 1.2 The Dynamic Discrete Choice Model

### State Variables

At each period $t$, a bank is characterized by state vector $s_t$:
- **Equity ratio** ($equity\_a$): Capital buffer against losses
- **Non-performing loans ratio** ($npf\_a$): Measure of asset quality
- **Log assets** ($\log(assets)$): Bank size
- **Real estate owned ratio** ($realest\_a$): Foreclosed properties
- **Return on assets** ($roa$): Profitability
- **Political indices** ($House$, $Senate$): Regulatory environment

### Value Functions

The regulator's value function for keeping the bank **open**:

$$V_{\text{open}}(s_t) = MC(s_t) + \beta \cdot \mathbb{E}_t[V(s_{t+1})]$$

where:
- $MC(s_t)$ = Net monetary cost of operating the bank (negative if bank is profitable)
- $\beta$ = Discount factor
- $\mathbb{E}_t[V(s_{t+1})]$ = Expected continuation value

The value of **closing** the bank: $V_{\text{close}}(s_t) = 0$ (normalized)

### Choice Probabilities

With Type-I Extreme Value (logit) errors, the probability of closing is:

$$P(\text{close} | s_t) = \frac{1}{1 + \exp\left(\frac{V_{\text{open}}(s_t) - V_{\text{close}}(s_t)}{\sigma}\right)}$$

## 1.3 The CCP Approach (Hotz-Miller 1993)

### The Inversion Formula

From the logit formula:

$$\sigma \cdot \log\left(\frac{1 - P_t}{P_t}\right) = V_{\text{open}}(s_t) - V_{\text{close}}(s_t)$$

### The Euler Equation / Moment Condition

Substituting the value function:

$$g(\theta) = \sigma \cdot \log\left(\frac{1-P_t}{P_t}\right) + \beta\sigma \cdot \mathbb{E}[\ln P_{t+1}] - \beta \cdot \mathbb{E}[MC_{t+1}] + MC_t$$

### Structural Parameters (10 total)

- $\beta$: Discount factor (quarterly)
- $\sigma$: Scale of idiosyncratic closure cost
- $\beta_{NMC}$: Coefficients of the net monetary cost function (8 parameters)

---

# Setup: Import Libraries

In [2]:
# Core scientific computing
import numpy as np
import pandas as pd
from scipy import stats
from scipy.optimize import minimize
from scipy.special import expit  # Logistic function
import scipy.io as sio
from tabulate import tabulate

# Visualization
import matplotlib.pyplot as plt

# Utilities
import warnings
warnings.filterwarnings('ignore')

# Set display options
pd.set_option('display.max_columns', 50)
pd.set_option('display.float_format', '{:.6f}'.format)
np.set_printoptions(precision=6, suppress=True)

# Plotting style
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 12

print("Libraries loaded successfully.")

Libraries loaded successfully.


---

# 2. Data Overview

## 2.1 Data Sources

The data comes from the **Call Reports** (quarterly regulatory filings by U.S. banks) covering:
- **Period**: 1985Q4 to 1992Q4 (the S&L Crisis era)
- **Sample**: 22,269 bank-quarter observations
- **Bank closures**: 714 failures (3.21%)

## 2.2 Pre-computed Data Files

This replication uses pre-computed files from the original MATLAB estimation:

| File | Contents | Stage |
|------|----------|-------|
| `dataforlogit_KLW.txt` | B-spline design matrix (22,269 x 80) | First stage |
| `logit_beta_KLW.txt` | Pre-computed logit coefficients (79) | First stage |
| `ccpnext_param_sim5000_KLW.txt` | Forward simulations (5,000 draws) | Second stage |
| `dataforparam_KLW.mat` | Merged GMM estimation data | Third stage |
| `theta_KLW.mat` | Original structural estimates | Verification |

**Note on Replication**: The first and second stages are NOT independently replicated here. We use pre-computed files because:
1. First-stage logit requires MATLAB's CompEcon B-spline toolbox
2. Second-stage simulation requires AR(4) transition parameter estimates from Stata

In [3]:
# Define paths
DATA_PATH = './data/'
OUTPUT_PATH = './output/'

print("="*70)
print("LOADING PRE-COMPUTED DATA FILES")
print("="*70)

# ============================================================
# Load the pre-computed logit design matrix
# ============================================================
print("\n1. Loading logit design matrix (B-spline basis functions)...")
logit_data = np.loadtxt(DATA_PATH + 'dataforlogit_KLW.txt')
print(f"   Shape: {logit_data.shape}")
print(f"   Columns 1-72: B-spline basis (6 vars x 4 lags x 3 basis)")
print(f"   Columns 73-76: Unemployment (4 lags)")
print(f"   Columns 77-78: House, Senate indices")
print(f"   Column 79: yearq")
print(f"   Column 80: failed (outcome)")

X_logit = logit_data[:, :78]  # First 78 columns are regressors
y_logit = logit_data[:, 79]   # Column 80 is outcome (0-indexed: 79)

print(f"\n   Number of failures: {int(y_logit.sum())} ({100*y_logit.mean():.2f}%)")

# ============================================================
# Load pre-computed forward simulations
# ============================================================
print("\n2. Loading forward simulation results (5,000 draws per obs)...")
ccp_next = np.loadtxt(OUTPUT_PATH + 'ccpnext_param_sim5000_KLW.txt')
print(f"   Shape: {ccp_next.shape}")

# ============================================================
# Load original logit coefficients
# ============================================================
print("\n3. Loading original logit coefficients (from Stata)...")
with open(OUTPUT_PATH + 'logit_beta_KLW.txt', 'r') as f:
    lines = f.readlines()
orig_logit_beta = []
for line in lines[3:]:
    parts = line.strip().split('\t')
    if len(parts) >= 2:
        try:
            orig_logit_beta.append(float(parts[1]))
        except:
            pass
orig_logit_beta = np.array(orig_logit_beta)
print(f"   Number of coefficients: {len(orig_logit_beta)}")

# ============================================================
# Load GMM estimation data
# ============================================================
print("\n4. Loading GMM estimation data...")
gmm_mat = sio.loadmat(OUTPUT_PATH + 'dataforparam_KLW.mat')
print(f"   Variables loaded: {len([k for k in gmm_mat.keys() if not k.startswith('__')])}")

# ============================================================
# Load original structural estimates for comparison
# ============================================================
print("\n5. Loading original structural estimates...")
theta_mat = sio.loadmat(OUTPUT_PATH + 'theta_KLW.mat')
theta_original = theta_mat['theta_KLW'].squeeze()
print(f"   Original theta: {theta_original}")

print("\n" + "="*70)
print("DATA LOADING COMPLETE")
print("="*70)

LOADING PRE-COMPUTED DATA FILES

1. Loading logit design matrix (B-spline basis functions)...
   Shape: (22269, 80)
   Columns 1-72: B-spline basis (6 vars x 4 lags x 3 basis)
   Columns 73-76: Unemployment (4 lags)
   Columns 77-78: House, Senate indices
   Column 79: yearq
   Column 80: failed (outcome)

   Number of failures: 714 (3.21%)

2. Loading forward simulation results (5,000 draws per obs)...
   Shape: (22269, 15)

3. Loading original logit coefficients (from Stata)...
   Number of coefficients: 79

4. Loading GMM estimation data...
   Variables loaded: 34

5. Loading original structural estimates...
   Original theta: [      0.958234     639.844059 -507234.992222   93645.094123
   -4320.306837  -54111.280274  -48971.264904 -183782.202034
    -829.006435     210.188013]

DATA LOADING COMPLETE


## 2.3 Summary Statistics

In [4]:
# =============================================================================
# Table 1: Summary Statistics (1985-1992)
# Replicating Table 1 from Kang, Lowery & Wardlaw (RFS 2015), page 16
# =============================================================================

print("="*75)
print("Table 1: Summary Statistics: 1985-1992")
print("="*75)

# Extract key variables from GMM data
equity_a = gmm_mat['equity_a_all'][:, 0].squeeze()
npf_a = gmm_mat['npf_a_all'][:, 0].squeeze()
log_assets = gmm_mat['logasset_all'][:, 0].squeeze()
roa = gmm_mat['roa_annual_all'][:, 0].squeeze()
realest = gmm_mat['realEst_own_a_all'][:, 0].squeeze()
unemp = gmm_mat['unemp_all'][:, 0].squeeze()
House = gmm_mat['House'].squeeze()
Senate = gmm_mat['Senate'].squeeze()
Bclose = gmm_mat['Bclose'].squeeze()
assets_M = np.exp(log_assets) / 1e6  # Convert to millions

# Panel A: Quarterly Call Report data/macroeconomic and political conditions
print("\nPanel A: Quarterly Call Report data/macroeconomic and political conditions")
print("-"*75)

def format_stat(val, decimals=3):
    """Format statistic with appropriate precision."""
    if abs(val) >= 100:
        return f"{val:.0f}"
    elif abs(val) >= 10:
        return f"{val:.1f}"
    else:
        return f"{val:.{decimals}f}"

panel_a_vars = [
    ('Assets ($M)', assets_M),
    ('Equity/assets', equity_a),
    ('Nonperforming loans/assets', npf_a),
    ('Real estate owned/assets', realest),
    ('Net income/assets', roa),
    ('State unemployment rate (%)', unemp),
    ('House', House),
    ('Senate', Senate),
]

panel_a_rows = []
for name, arr in panel_a_vars:
    panel_a_rows.append([
        name,
        format_stat(arr.mean()),
        format_stat(arr.std()),
        format_stat(np.percentile(arr, 5)),
        format_stat(np.median(arr)),
        format_stat(np.percentile(arr, 95))
    ])

print(tabulate(panel_a_rows,
               headers=['', 'Mean', 'Sd', '5%', 'Median', '95%'],
               tablefmt='simple',
               colalign=('left', 'right', 'right', 'right', 'right', 'right')))

print(f"\nObservations: {len(equity_a):,}")
print(f"Unique banks: {len(np.unique(gmm_mat['BankID'])):,}")

# Panel B: FDIC failure and merger data
print("\n" + "-"*75)
print("Panel B: FDIC failure and merger data")
print("-"*75)

n_failures = int(Bclose.sum())
failure_rate = 100 * Bclose.mean()

print(f"\nFailures: {n_failures}")
print(f"Failure rate: {failure_rate:.2f}%")

print("\n" + "="*75)
print("Note: This table corresponds to Table 1 in the paper (page 16).")
print("="*75)

Table 1: Summary Statistics: 1985-1992

Panel A: Quarterly Call Report data/macroeconomic and political conditions
---------------------------------------------------------------------------
                               Mean     Sd      5%    Median     95%
---------------------------  ------  -----  ------  --------  ------
Assets ($M)                   0.125  0.238   0.016     0.052    0.51
Equity/assets                 0.054  0.039       0     0.057   0.106
Nonperforming loans/assets    0.041  0.036   0.003     0.031   0.108
Real estate owned/assets      0.026   0.03       0     0.017   0.083
Net income/assets            -0.022  0.024  -0.065    -0.014  -0.001
State unemployment rate (%)    6.96  1.694     4.5       6.7     9.6
House                         3.148  2.903       0         3      10
Senate                        1.423  0.757       0         2       2

Observations: 22,269
Unique banks: 4,661

---------------------------------------------------------------------------


In [ ]:
# Visualize key variables
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# Capital ratio distribution
axes[0, 0].hist(equity_a, bins=50, edgecolor='black', alpha=0.7)
axes[0, 0].axvline(x=0.08, color='red', linestyle='--', linewidth=2, label='Regulatory min (8%)')
axes[0, 0].set_xlabel('Equity / Assets')
axes[0, 0].set_ylabel('Frequency')
axes[0, 0].set_title('Capital Ratio Distribution')
axes[0, 0].legend()

# NPL distribution
axes[0, 1].hist(npf_a, bins=50, edgecolor='black', alpha=0.7)
axes[0, 1].set_xlabel('Non-Performing Loans / Assets')
axes[0, 1].set_ylabel('Frequency')
axes[0, 1].set_title('Asset Quality Distribution')

# Compare failed vs surviving: equity
failed_mask = Bclose == 1
axes[1, 0].hist(equity_a[~failed_mask], bins=30, alpha=0.5, label='Surviving', density=True)
axes[1, 0].hist(equity_a[failed_mask], bins=30, alpha=0.5, label='Failed', density=True)
axes[1, 0].set_xlabel('Equity / Assets')
axes[1, 0].set_ylabel('Density')
axes[1, 0].set_title('Capital: Failed vs Surviving Banks')
axes[1, 0].legend()

# Compare failed vs surviving: NPL
axes[1, 1].hist(npf_a[~failed_mask], bins=30, alpha=0.5, label='Surviving', density=True)
axes[1, 1].hist(npf_a[failed_mask], bins=30, alpha=0.5, label='Failed', density=True)
axes[1, 1].set_xlabel('Non-Performing Loans / Assets')
axes[1, 1].set_ylabel('Density')
axes[1, 1].set_title('Asset Quality: Failed vs Surviving Banks')
axes[1, 1].legend()

plt.tight_layout()
plt.show()

---

# 3. First Stage: CCP Estimation

## 3.1 Flexible Logit Specification

The first stage estimates conditional choice probabilities (CCPs) using a flexible logit model.

### Why B-spline Basis Functions?

We avoid imposing restrictive functional forms by using **B-spline basis functions**:
- 6 state variables (equity, NPL, log_assets, real_estate, ROA, asset_growth)
- 4 lags each (current + 3 lags)
- 3 B-spline basis functions per variable-lag
- Total: 72 basis + 4 unemployment lags + 2 political indices + 1 constant = **79 parameters**

### Note on Replication

**We use pre-computed logit coefficients from the original Stata estimation** because:
1. The B-spline basis requires MATLAB's CompEcon toolbox (not available in Python)
2. The high-dimensional logit (79 parameters) requires careful regularization

The logit model is:
$$P(\text{close}_t = 1 | s_t) = \Lambda(X_t \beta) = \frac{1}{1 + \exp(-X_t \beta)}$$

In [ ]:
print("="*70)
print("FIRST STAGE: CCP Estimation")
print("="*70)

# Add constant to design matrix
X_logit_const = np.column_stack([X_logit, np.ones(len(X_logit))])
print(f"\nDesign matrix with constant: {X_logit_const.shape}")
print(f"Number of parameters: {X_logit_const.shape[1]}")

## 3.2 Using Pre-computed Coefficients

The original estimation was performed in **Stata** (see `logit_KLW.do`):
```stata
logit failed lnbasis_1 - lnbasis_72 unemp* house senate
```

We load and verify the pre-computed coefficients.

In [ ]:
print("\n--- Using Pre-computed Logit Coefficients ---")

# Compute predictions using original betas
P_hat = expit(X_logit_const @ orig_logit_beta)

# Load original predictions for verification
P_hat_file = np.loadtxt(OUTPUT_PATH + 'logit_predict_KLW.txt')

print(f"Predictions using original betas: mean={P_hat.mean():.6f}")
print(f"Original prediction file:         mean={P_hat_file.mean():.6f}")
print(f"Correlation: {np.corrcoef(P_hat, P_hat_file)[0,1]:.8f}")
print(f"Max absolute difference: {np.abs(P_hat - P_hat_file).max():.10f}")

if np.corrcoef(P_hat, P_hat_file)[0,1] > 0.9999:
    print("\n>>> VERIFICATION PASSED: Predictions match original!")

## 3.3 Verification and Diagnostics

In [ ]:
# Use original predictions for subsequent analysis
P_hat = P_hat_file.copy()

print("Using ORIGINAL logit predictions for exact replication.")
print(f"\nPredicted closure probabilities:")
print(f"  Min:  {P_hat.min():.8f}")
print(f"  Max:  {P_hat.max():.8f}")
print(f"  Mean: {P_hat.mean():.8f}")
print(f"  Actual closure rate: {y_logit.mean():.8f}")

In [ ]:
# Diagnostic plots
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Distribution of predicted probabilities
axes[0].hist(P_hat[y_logit == 0], bins=50, alpha=0.5, label='Surviving', density=True)
axes[0].hist(P_hat[y_logit == 1], bins=50, alpha=0.5, label='Failed', density=True)
axes[0].set_xlabel('Predicted P(Close)')
axes[0].set_ylabel('Density')
axes[0].set_title('Distribution of Predicted Closure Probabilities')
axes[0].legend()

# Calibration plot
bins = np.linspace(0, 1, 21)
bin_indices = np.digitize(P_hat, bins)
bin_means = []
actual_rates = []
for i in range(1, len(bins)):
    mask = bin_indices == i
    if mask.sum() > 0:
        bin_means.append(P_hat[mask].mean())
        actual_rates.append(y_logit[mask].mean())

axes[1].scatter(bin_means, actual_rates, s=100, alpha=0.7)
axes[1].plot([0, 1], [0, 1], 'r--', label='Perfect calibration')
axes[1].set_xlabel('Mean Predicted Probability')
axes[1].set_ylabel('Actual Closure Rate')
axes[1].set_title('Calibration Plot')
axes[1].legend()

plt.tight_layout()
plt.show()

---

# 4. Second Stage: Forward Simulation

## 4.1 State Transition Model

The moment condition requires $\mathbb{E}_t[\ln P(\text{close}_{t+1} | s_{t+1})]$, the expected log probability of closure next period.

### AR(4) Transition Processes

Each state variable follows an AR(4) process estimated from the data:
$$x_{t+1} = \alpha_0 + \alpha_1 x_t + \alpha_2 x_{t-1} + \alpha_3 x_{t-2} + \alpha_4 x_{t-3} + \varepsilon_{t+1}$$

## 4.2 Monte Carlo Integration

For each observation, the MATLAB code (`ccpnext_param_KLW.m`):
1. Draws 5,000 realizations from the empirical residual distribution (winsorized to [0.25, 99.75] percentiles)
2. Computes next-period state for each draw
3. Evaluates the logit CCP at the simulated state
4. Averages $\ln P_{t+1}$ and $MC_{t+1}$ across draws

### Note on Replication

**We use pre-computed forward simulation results** because:
1. The procedure requires AR(4) transition coefficients estimated in Stata
2. Reproducing the exact random number sequence requires MATLAB's RNG seed

In [ ]:
print("="*70)
print("SECOND STAGE: Forward Simulation Results")
print("="*70)

print(f"\nPre-computed simulation results: {ccp_next.shape}")
print(f"Number of observations: {ccp_next.shape[0]:,}")
print(f"Number of simulation draws per obs: 5,000")
print(f"Total simulations: {ccp_next.shape[0] * 5000:,}")

# Extract key columns
E_P_next = ccp_next[:, 2]      # E[P(close_{t+1})]
E_ln_P_next = ccp_next[:, 3]   # E[ln P(close_{t+1})]
E_MC_next_sim = ccp_next[:, 4] # E[MC_{t+1}]

print(f"\n--- E[P_next] Statistics ---")
print(f"  Range: [{E_P_next.min():.8f}, {E_P_next.max():.8f}]")
print(f"  Mean:  {E_P_next.mean():.8f}")

print(f"\n--- E[ln P_next] Statistics ---")
print(f"  Range: [{E_ln_P_next.min():.4f}, {E_ln_P_next.max():.4f}]")
print(f"  Mean:  {E_ln_P_next.mean():.4f}")

## 4.3 Verification

In [ ]:
# Verify consistency with GMM data
print("\n--- Verification Against GMM Data ---")

E_ln_P_gmm = gmm_mat['Eln_prob_next'].squeeze()

print(f"From ccpnext file:   E[ln P] mean = {E_ln_P_next.mean():.6f}")
print(f"From GMM .mat file:  E[ln P] mean = {E_ln_P_gmm.mean():.6f}")
print(f"Correlation: {np.corrcoef(E_ln_P_next, E_ln_P_gmm)[0,1]:.8f}")

if np.corrcoef(E_ln_P_next, E_ln_P_gmm)[0,1] > 0.9999:
    print("\n>>> VERIFICATION PASSED: Forward simulations match!")

---

# 5. Third Stage: Structural GMM Estimation

## 5.1 The Moment Condition

From the CCP inversion and Euler equation:

$$g(\theta) = \sigma \cdot \ln\left(\frac{1-P_t}{P_t}\right) + \beta\sigma \cdot \mathbb{E}[\ln P_{t+1}] - \beta \cdot \left(-\mathbb{E}[MC_{t+1}] + X_{t+1}\beta_{NMC}\right) + \left(-MC_t + X_t \beta_{NMC}\right)$$

### The Net Monetary Cost (NMC) Function

$$NMC(s_t) = -MC(s_t) + X_t \beta_{NMC}$$

where $MC(s_t)$ is the pre-computed monetary cost from a Tobit model, and $X_t \beta_{NMC}$ is a linear adjustment.

**Note**: The pre-computed $MC$ already captures the direct cost component; $\beta_{NMC}$ captures additional effects through the listed covariates.

## 5.2 Continuously-Updated GMM

**CRITICAL**: The original uses **continuously-updated GMM**, not standard two-step GMM.

Standard GMM: $W = (Z'Z/N)^{-1}$ (fixed instruments)

**Continuously-updated GMM**: $W = (G'G/N)^{-1}$ (moment contributions)

From `param_obj_cont_KLW.m` (lines 66-70):
```matlab
G = [g1,g2,g3,g4,g5,g6,g7,g8,g9,g10,g11];
W = (1/Nobs)*(G'*G);
f_temp = W\m';
f = m*f_temp;
```

### Instruments (11 moments)

1. Constant
2. log(assets)
3. log(assets)$^2$
4. NPL ratio
5. ROA
6. House index
7. Senate index
8. Equity ratio
9. Real estate ratio
10. Asset growth
11. State unemployment

In [ ]:
print("="*70)
print("THIRD STAGE: Structural GMM Estimation")
print("="*70)

# ============================================================
# Extract all data from GMM .mat file
# ============================================================

# CCPs
Eprob_now = gmm_mat['Eprob_now'].squeeze()
Eln_prob_next = gmm_mat['Eln_prob_next'].squeeze()

# Monetary costs (from pre-estimated Tobit model)
Emc_now = gmm_mat['Emc_now'].squeeze()
Emc_next = gmm_mat['Emc_next'].squeeze()

# Political indices
House = gmm_mat['House'].squeeze()
Senate = gmm_mat['Senate'].squeeze()
House_next = gmm_mat['House_next'].squeeze()
Senate_next = gmm_mat['Senate_next'].squeeze()

# State variables
logasset_all = gmm_mat['logasset_all']
npf_a_all = gmm_mat['npf_a_all']
roa_annual_all = gmm_mat['roa_annual_all']
realEst_own_a_all = gmm_mat['realEst_own_a_all']
equity_a_all = gmm_mat['equity_a_all']
assGrowth_1y_all = gmm_mat['assGrowth_1y_all']
unemp_all = gmm_mat['unemp_all']

# Expected next-period variables
Elogasset_next = gmm_mat['Elogasset_next'].squeeze()
Elogasset_next2 = gmm_mat['Elogasset_next2'].squeeze()
Enpf_a_next = gmm_mat['Enpf_a_next'].squeeze()
Erealest_a_next = gmm_mat['Erealest_a_next'].squeeze()
Eroa_next = gmm_mat['Eroa_next'].squeeze()

Nobs = len(Eprob_now)
ONES = np.ones(Nobs)

print(f"\nSample size for GMM: {Nobs:,} observations")

In [ ]:
# ============================================================
# Handle extreme probabilities (as in original code)
# ============================================================

print("\n--- Handling Extreme Probabilities ---")
print(f"Max P before adjustment: {Eprob_now.max():.10f}")

# From param_obj_cont_KLW.m lines 27-30:
# One observation has P very close to 1, causing log(1-P) issues
sorted_idx = np.argsort(Eprob_now)
idx_biggest = sorted_idx[-1]
idx_2biggest = sorted_idx[-2]

Eprob_now_adj = Eprob_now.copy()
Eprob_now_adj[idx_biggest] = Eprob_now[idx_2biggest] + (1 - Eprob_now[idx_2biggest]) * 0.9

print(f"Max P after adjustment:  {Eprob_now_adj.max():.10f}")
print(f"Adjustment applied to observation {idx_biggest}")

In [ ]:
# ============================================================
# Construct design matrices (exactly as in param_obj_KLW.m)
# ============================================================

print("\n--- Constructing Design Matrices ---")

# X_now = [const, log_assets, log_assets^2, npf_a, roa, realest_a, House, Senate]
Xnow = np.column_stack([
    ONES,
    logasset_all[:, 0],
    logasset_all[:, 0]**2,
    npf_a_all[:, 0],
    roa_annual_all[:, 0],
    realEst_own_a_all[:, 0],
    House,
    Senate
])

# X_next = [const, E[log_assets], E[log_assets^2], E[npf], E[roa], E[realest], House_next, Senate_next]
Xnext = np.column_stack([
    ONES,
    Elogasset_next,
    Elogasset_next2,
    Enpf_a_next,
    Eroa_next,
    Erealest_a_next,
    House_next,
    Senate_next
])

# Instruments Z (11 columns)
Z = np.column_stack([
    ONES,
    logasset_all[:, 0],
    logasset_all[:, 0]**2,
    npf_a_all[:, 0],
    roa_annual_all[:, 0],
    House,
    Senate,
    equity_a_all[:, 0],
    realEst_own_a_all[:, 0],
    assGrowth_1y_all[:, 0],
    unemp_all[:, 0]
])

print(f"X_now shape:  {Xnow.shape}")
print(f"X_next shape: {Xnext.shape}")
print(f"Z shape:      {Z.shape}")

In [ ]:
# ============================================================
# GMM Objective Function: CONTINUOUSLY-UPDATED GMM
# Matches param_obj_cont_KLW.m exactly
# ============================================================

def gmm_objective_cont(THETA, return_details=False):
    """
    Continuously-updated GMM objective function.
    
    CRITICAL: Uses W = G'G (moment contributions), NOT W = Z'Z (instruments)
    
    Parameters:
    -----------
    THETA : array of length 10
        [beta, sigma, beta_NMC[0:8]]
    return_details : bool
        If True, return (f, m, G) for debugging
    
    Returns:
    --------
    f : float
        GMM objective value (to be minimized)
    """
    beta = THETA[0]       # Discount factor
    sigma = THETA[1]      # Scale parameter
    beta_NMC = THETA[2:10]  # NMC coefficients
    
    # Compute log-odds: log((1-P)/P)
    log_odds = np.log(1 - Eprob_now_adj) - np.log(Eprob_now_adj)
    
    # Moment condition g1 (equation from CCP inversion)
    g1 = (sigma * log_odds + 
          beta * sigma * Eln_prob_next - 
          beta * (-Emc_next + Xnext @ beta_NMC) + 
          (-Emc_now + Xnow @ beta_NMC))
    
    # Interact base moment with instruments
    g2 = g1 * Z[:, 1]
    g3 = g1 * Z[:, 2]
    g4 = g1 * Z[:, 3]
    g5 = g1 * Z[:, 4]
    g6 = g1 * Z[:, 5]
    g7 = g1 * Z[:, 6]
    g8 = g1 * Z[:, 7]
    g9 = g1 * Z[:, 8]
    g10 = g1 * Z[:, 9]
    g11 = g1 * Z[:, 10]
    
    # Sample moment vector
    m = np.array([
        g1.mean(), g2.mean(), g3.mean(), g4.mean(), g5.mean(),
        g6.mean(), g7.mean(), g8.mean(), g9.mean(), g10.mean(), g11.mean()
    ])
    
    # CONTINUOUSLY-UPDATED WEIGHTING: W = G'G (not Z'Z!)
    G = np.column_stack([g1, g2, g3, g4, g5, g6, g7, g8, g9, g10, g11])
    W = (G.T @ G) / Nobs
    
    # GMM objective: m' * W^{-1} * m
    try:
        f_temp = np.linalg.solve(W, m)
        f = m @ f_temp
    except np.linalg.LinAlgError:
        f = 1e10
    
    if return_details:
        return f, m, G
    return f

print("GMM objective function defined (continuously-updated weighting).")

In [ ]:
# ============================================================
# Verify objective function at original estimates
# ============================================================

print("Verification: Objective at Original Estimates")
print("="*50)

param_names = ['beta', 'sigma', 'NMC:const', 'NMC:logA', 'NMC:logA2', 
               'NMC:npf', 'NMC:roa', 'NMC:realest', 'NMC:House', 'NMC:Senate']

# Display original theta using tabulate
orig_table = [[name, f"{val:.4f}"] for name, val in zip(param_names, theta_original)]
print("\nOriginal estimates:")
print(tabulate(orig_table, headers=['Parameter', 'Value'], tablefmt='simple'))

f_at_original = gmm_objective_cont(theta_original)
print(f"\nGMM objective at original: {f_at_original:.8f}")

# Check gradient (should be near zero at optimum)
eps = 1e-6
grad = np.zeros(10)
for i in range(10):
    theta_plus = theta_original.copy()
    theta_minus = theta_original.copy()
    theta_plus[i] += eps
    theta_minus[i] -= eps
    grad[i] = (gmm_objective_cont(theta_plus) - gmm_objective_cont(theta_minus)) / (2 * eps)

print(f"Gradient norm at original: {np.linalg.norm(grad):.8f}")

if f_at_original < 0.001 and np.linalg.norm(grad) < 0.001:
    print("\n>>> VERIFICATION PASSED: Original estimates are at a local minimum!")

## 5.3 Grid-Search Refinement Procedure

**Key Insight from the Original Code**: Standard gradient-based optimization converges to different local minima depending on starting values. The original MATLAB code uses an **iterative grid search refinement** procedure.

From `fn_obj_cont_KLW.m` (lines 76-132) and `findmin_cont_KLW.m` (lines 26-60):

1. Run gradient-based optimization (fmincon)
2. For each parameter, search over a grid of $\pm 100\%$ of its current value
3. Find the parameter values that minimize the objective for each parameter
4. Compute objective at the grid-refined parameter vector
5. If grid search finds a lower objective, use those parameters
6. Repeat until convergence (up to 25 iterations)

In [ ]:
# ============================================================
# Grid-Search Refinement (from fn_obj_cont_KLW.m)
# ============================================================

def grid_search_refinement(THETA, num_range=100, portion=0.01, extend=1.0):
    """
    Grid search refinement procedure from fn_obj_cont_KLW.m.
    
    For each parameter, searches over a grid from:
        theta[i] - theta[i]*portion*num_range  to  theta[i] + theta[i]*portion*num_range*extend
    
    Parameters:
    -----------
    THETA : array of length 10
        Current parameter estimates
    num_range : int
        Number of grid points on each side (default 100)
    portion : float
        Step size as fraction of parameter (default 0.01 = 1%)
    extend : float
        Extension factor for positive side (default 1.0)
    
    Returns:
    --------
    THETA_new : array
        Refined parameter estimates
    f_new : float
        Objective at refined estimates
    improved : bool
        Whether refinement improved the objective
    """
    f_old = gmm_objective_cont(THETA)
    
    # For each parameter, find the grid value that minimizes objective (holding others fixed)
    THETA_new = np.zeros_like(THETA)
    
    for i_param in range(len(THETA)):
        # Create grid for this parameter
        theta_i = THETA[i_param]
        if abs(theta_i) < 1e-10:  # Handle near-zero parameters
            theta_range = np.linspace(-1, 1, 2*num_range + 1)
        else:
            theta_range = np.linspace(
                theta_i - abs(theta_i) * portion * num_range,
                theta_i + abs(theta_i) * portion * num_range * extend,
                2 * num_range + 1
            )
        
        # Evaluate objective at each grid point
        f_range = np.zeros(len(theta_range))
        for k, theta_k in enumerate(theta_range):
            THETA_temp = THETA.copy()
            THETA_temp[i_param] = theta_k
            f_range[k] = gmm_objective_cont(THETA_temp)
        
        # Find minimizing grid point
        best_idx = np.argmin(f_range)
        THETA_new[i_param] = theta_range[best_idx]
    
    f_new = gmm_objective_cont(THETA_new)
    
    return THETA_new, f_new, f_new < f_old

print("Grid search refinement function defined.")

In [ ]:
# ============================================================
# Full Optimization with Grid Refinement (from findmin_cont_KLW.m)
# ============================================================

def gmm_optimize_with_grid_refinement(x0, max_iterations=25, tol=1e-8, verbose=True):
    """
    GMM optimization with iterative grid search refinement.
    Matches the procedure in findmin_cont_KLW.m.
    
    Parameters:
    -----------
    x0 : array
        Starting values
    max_iterations : int
        Maximum number of refinement iterations
    tol : float
        Convergence tolerance
    verbose : bool
        Print progress
    
    Returns:
    --------
    theta_best : array
        Best parameter estimates found
    f_best : float
        Objective at best estimates
    history : list
        History of objective values
    """
    # Bounds
    bounds = [
        (0.001, 0.999),  # beta
        (0.001, None),   # sigma
    ] + [(None, None)] * 8
    
    theta_current = x0.copy()
    f_current = gmm_objective_cont(theta_current)
    history = [f_current]
    
    if verbose:
        print(f"\nStarting optimization with grid refinement")
        print(f"Initial objective: {f_current:.8f}")
        print("-" * 60)
    
    for iteration in range(max_iterations):
        # Step 1: Gradient-based optimization
        result = minimize(
            gmm_objective_cont,
            theta_current,
            method='L-BFGS-B',
            bounds=bounds,
            options={'maxiter': 500, 'ftol': 1e-12}
        )
        theta_grad = result.x
        f_grad = result.fun
        
        # Step 2: Grid search refinement
        theta_grid, f_grid, improved = grid_search_refinement(theta_grad)
        
        # Choose better result
        if f_grid < f_grad:
            theta_new = theta_grid
            f_new = f_grid
            method_used = "grid"
        else:
            theta_new = theta_grad
            f_new = f_grad
            method_used = "gradient"
        
        history.append(f_new)
        
        if verbose:
            print(f"Iter {iteration+1:2d}: f={f_new:.8f} (from {method_used}), improvement={f_current - f_new:.2e}")
        
        # Check convergence
        if abs(f_current - f_new) < tol:
            if verbose:
                print(f"\nConverged after {iteration+1} iterations.")
            break
        
        theta_current = theta_new
        f_current = f_new
    
    return theta_current, f_current, history

print("Full optimization procedure defined.")

## 5.4 Estimation Results

In [ ]:
# ============================================================
# Run optimization from initial values
# ============================================================

print("="*70)
print("GMM ESTIMATION WITH GRID REFINEMENT")
print("="*70)

# Starting values (from GMM1st_KLW.mat - first-stage GMM estimates)
x_initial = np.array([
    9.54192957e-01,   # beta
    5.77371682e+02,   # sigma
    -4.68922494e+05,  # NMC_const
    8.56032237e+04,   # NMC_logA
    -3.91390922e+03,  # NMC_logA2
    -3.38245568e+04,  # NMC_npf
    -5.48256731e+04,  # NMC_roa
    -1.46755533e+05,  # NMC_realest
    -4.19949688e+02,  # NMC_House
    1.13845592e+01    # NMC_Senate
])

print(f"\nStarting from first-stage GMM estimates")
print(f"Initial objective: {gmm_objective_cont(x_initial):.8f}")

# Run optimization with grid refinement
theta_estimated, f_estimated, history = gmm_optimize_with_grid_refinement(
    x_initial, max_iterations=15, verbose=True
)

In [ ]:
# ============================================================
# Also run from original estimates to verify
# ============================================================

print("\n" + "="*70)
print("VERIFICATION: Starting from Original Estimates")
print("="*70)

theta_from_orig, f_from_orig, history_orig = gmm_optimize_with_grid_refinement(
    theta_original, max_iterations=5, verbose=True
)

In [ ]:
# ============================================================
# Compare results
# ============================================================

print("Table: Replication Comparison")
print("="*60)

# Use the result with lower objective
if f_estimated < f_from_orig:
    theta_final = theta_estimated
    f_final = f_estimated
    source = "initial values"
else:
    theta_final = theta_from_orig
    f_final = f_from_orig
    source = "original"

print(f"\nUsing estimates from {source} (lower objective)")

# Create comparison table using tabulate
comparison_data = []
for i, name in enumerate(param_names):
    orig = theta_original[i]
    est = theta_final[i]
    pct_diff = 100 * (est - orig) / abs(orig) if orig != 0 else 0
    comparison_data.append([name, f"{orig:.4f}", f"{est:.4f}", f"{pct_diff:.2f}%"])

print("\n" + tabulate(comparison_data, 
                      headers=['Parameter', 'Original', 'Replicated', '% Diff'],
                      tablefmt='simple',
                      colalign=('left', 'right', 'right', 'right')))

print(f"\nGMM Objective:")
print(f"  Original:   {gmm_objective_cont(theta_original):.8f}")
print(f"  Replicated: {f_final:.8f}")

In [ ]:
# Plot convergence
fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(history, 'b-o', label='From initial values')
ax.axhline(y=gmm_objective_cont(theta_original), color='r', linestyle='--', 
           label=f'Original objective ({gmm_objective_cont(theta_original):.6f})')
ax.set_xlabel('Iteration')
ax.set_ylabel('GMM Objective')
ax.set_title('Convergence of GMM Estimation with Grid Refinement')
ax.legend()
ax.set_yscale('log')
plt.tight_layout()
plt.show()

---

# 6. Inference and Results

## 6.1 Standard Error Computation

For continuously-updated GMM, the asymptotic variance has the form:

$$\text{Avar}(\hat{\theta}) = \frac{1}{N} (\Gamma' W^{-1} \Gamma)^{-1}$$

where $\Gamma = \partial m / \partial \theta$ is the Jacobian of moments with respect to parameters.

## 6.2 Pre-Estimation Variance Adjustments

**Important**: The original code (`Avar_param_obj_KLW.m`) includes adjustments for pre-estimation error from:
1. First-stage logit CCP estimation
2. Tobit cost function estimation

Following Newey-McFadden (1994), the total variance is:

$$\text{Avar} = \text{Avar}_{\text{base}} + \text{Avar}_{\text{logit\_adj}} + \text{Avar}_{\text{cost\_adj}}$$

**Note**: The adjustments require the full covariance matrices from pre-estimation stages. We compute the base variance here and note that full standard errors would be slightly larger.

In [ ]:
# =============================================================================
# Table 7: Structural Parameters - Nonmonetary Costs
# Replicating Table 7, Column "Model I (85-92)" from KLW (RFS 2015), page 27
# =============================================================================

print("="*70)
print("Table 7: Structural Parameters: Nonmonetary Costs")
print("Model I (85-92) - Parsimonious specification, 1985-1992 sample")
print("="*70)

def compute_gmm_se(theta, eps=1e-5):
    """
    Compute standard errors for continuously-updated GMM.
    
    Uses numerical derivatives for the Jacobian.
    
    NOTE: This computes the BASE variance only. Full variance
    requires pre-estimation adjustments per Avar_param_obj_KLW.m.
    """
    n_params = len(theta)
    n_moments = 11
    
    def compute_moments(th):
        beta, sigma = th[0], th[1]
        beta_NMC = th[2:10]
        
        log_odds = np.log(1 - Eprob_now_adj) - np.log(Eprob_now_adj)
        g1 = (sigma * log_odds + 
              beta * sigma * Eln_prob_next - 
              beta * (-Emc_next + Xnext @ beta_NMC) + 
              (-Emc_now + Xnow @ beta_NMC))
        
        m = np.array([
            g1.mean(),
            (g1 * Z[:, 1]).mean(),
            (g1 * Z[:, 2]).mean(),
            (g1 * Z[:, 3]).mean(),
            (g1 * Z[:, 4]).mean(),
            (g1 * Z[:, 5]).mean(),
            (g1 * Z[:, 6]).mean(),
            (g1 * Z[:, 7]).mean(),
            (g1 * Z[:, 8]).mean(),
            (g1 * Z[:, 9]).mean(),
            (g1 * Z[:, 10]).mean()
        ])
        return m
    
    # Numerical Jacobian: Gamma = dm/dtheta
    Gamma = np.zeros((n_moments, n_params))
    for j in range(n_params):
        theta_plus = theta.copy()
        theta_minus = theta.copy()
        theta_plus[j] += eps
        theta_minus[j] -= eps
        Gamma[:, j] = (compute_moments(theta_plus) - compute_moments(theta_minus)) / (2 * eps)
    
    # Compute moment contributions at theta
    beta, sigma = theta[0], theta[1]
    beta_NMC = theta[2:10]
    log_odds = np.log(1 - Eprob_now_adj) - np.log(Eprob_now_adj)
    g1 = (sigma * log_odds + 
          beta * sigma * Eln_prob_next - 
          beta * (-Emc_next + Xnext @ beta_NMC) + 
          (-Emc_now + Xnow @ beta_NMC))
    
    G_mat = np.column_stack([
        g1, g1*Z[:,1], g1*Z[:,2], g1*Z[:,3], g1*Z[:,4],
        g1*Z[:,5], g1*Z[:,6], g1*Z[:,7], g1*Z[:,8], g1*Z[:,9], g1*Z[:,10]
    ])
    
    # Weighting matrix and its inverse
    W = (G_mat.T @ G_mat) / Nobs
    W_inv = np.linalg.inv(W)
    
    # Variance: (Gamma' W^{-1} Gamma)^{-1} / N
    bread = Gamma.T @ W_inv @ Gamma
    V = np.linalg.inv(bread) / Nobs
    
    se = np.sqrt(np.diag(V))
    return se, V

# Compute standard errors at final estimates
se, V = compute_gmm_se(theta_final)
t_stats = theta_final / se

# Helper for significance stars
def get_stars(t):
    if abs(t) > 2.58:
        return '***'
    elif abs(t) > 1.96:
        return '**'
    elif abs(t) > 1.65:
        return '*'
    return ''

# Parameter names matching Table 7 in paper (page 27)
table7_params = [
    ('Intercept', 2),          # NMC constant
    ('log(Assets)', 3),        # NMC log assets
    ('(log(Assets))^2', 4),    # NMC log assets squared
    ('NP Loans/Assets', 5),    # NMC NPL ratio
    ('Net Income/Assets', 6),  # NMC ROA
    ('RE Owned/Assets', 7),    # NMC real estate
    ('House', 8),              # NMC House
    ('Senate', 9),             # NMC Senate
    ('beta (quarterly discount factor)', 0),  # beta
    ('sigma', 1),              # sigma
]

# Build table rows
table_rows = []
for name, idx in table7_params:
    est = theta_final[idx]
    se_val = se[idx]
    stars = get_stars(t_stats[idx])
    table_rows.append([name, f"{est:.4f}{stars}", f"({se_val:.4f})"])

print(tabulate(table_rows,
               headers=['', 'Model I (85-92)', ''],
               tablefmt='simple',
               colalign=('left', 'right', 'left')))

# J-test
f_obj = gmm_objective_cont(theta_final)
n_moments = 11
n_params = 10
df = n_moments - n_params
j_stat = Nobs * f_obj
j_pval = 1 - stats.chi2.cdf(j_stat, df)

print(f"\nJ-test p-value                       {j_pval:.4f}")
print(f"Observations                         {Nobs:,}")

print("\n" + "-"*70)
print("Notes: Standard errors in parentheses. *** p<0.01, ** p<0.05, * p<0.10")
print("This replicates Model I (85-92) from Table 7 in the paper (page 27).")
print("Table 7 also contains Models I(a), II, III, IV, and V (08-12).")
print("-"*70)

---

# 7. Economic Interpretation

## 7.1 Parameter Interpretation

### Discount Factor ($\beta \approx 0.958$)

The quarterly discount factor implies:
- Annual discount factor: $\beta^4 \approx 0.843$
- Implied annual discount rate: $r \approx 15.7\%$

**Economic Interpretation**: This is substantially higher than typical market discount rates (5-10%). Possible explanations:
1. **Regulatory time pressure**: Political incentives to resolve problems quickly
2. **Agency costs**: Regulators may have shorter effective horizons than socially optimal
3. **Option value**: Waiting has opportunity cost when bank conditions deteriorate

### Scale Parameter ($\sigma \approx 640$)

The scale parameter captures heterogeneity in unobserved closure costs. A larger $\sigma$:
- Makes closure decisions **less deterministic** given observables
- Reflects idiosyncratic factors (local political pressure, examiner discretion, etc.)

In [ ]:
print("Economic Interpretation")
print("="*50)

beta_est = theta_final[0]
sigma_est = theta_final[1]

# Discount factor interpretation
print("\n1. DISCOUNT FACTOR")
discount_data = [
    ['Quarterly β', f"{beta_est:.4f}"],
    ['Annual β', f"{beta_est**4:.4f}"],
    ['Implied annual rate', f"{100*(1 - beta_est**4):.1f}%"]
]
print(tabulate(discount_data, tablefmt='simple'))
print("\n   Interpretation: Regulators discount future costs more heavily")
print("   than market rates, reflecting political pressures or agency costs.")

print("\n2. SCALE PARAMETER")
print(f"   σ = {sigma_est:.2f}")
print("\n   Interpretation: Substantial unobserved heterogeneity exists in")
print("   closure costs, including examiner discretion and local factors.")

## 7.2 Net Monetary Cost Function

The NMC function is:
$$NMC(s_t) = -MC(s_t) + X_t \beta_{NMC}$$

where $MC(s_t)$ is the pre-computed monetary cost from the Tobit model, and $X_t \beta_{NMC}$ captures additional effects.

### Interpreting the NMC Coefficients

| Coefficient | Estimate | Interpretation |
|-------------|----------|----------------|
| `NMC:const` | $< 0$ | Base adjustment is negative |
| `NMC:logA` | $> 0$ | Larger banks have higher adjusted NMC (more costly to keep open) |
| `NMC:logA2` | $< 0$ | Diminishing returns to size |
| `NMC:npf` | $< 0$ | Higher NPLs reduce adjusted NMC (NPL effect already in MC) |
| `NMC:roa` | $< 0$ | Higher profitability reduces adjusted NMC |
| `NMC:realest` | $< 0$ | Real estate effect (already in MC) |
| `NMC:House` | $< 0$ | Political index reduces closure costs |
| `NMC:Senate` | varies | Political index effect (often insignificant) |

**Important Caveat**: The NMC coefficients adjust a pre-computed $MC$ variable. Their signs reflect **residual effects** after the Tobit model, not the total effect of each variable on closure costs.

In [ ]:
print("Table: Net Monetary Cost (NMC) Coefficients")
print("="*60)
print("\nThese coefficients adjust the pre-computed monetary cost MC.")
print("Signs reflect RESIDUAL effects after the Tobit cost model.\n")

nmc_data = [
    ('Constant', theta_final[2], 'Base adjustment'),
    ('Log(assets)', theta_final[3], 'Larger banks → higher NMC'),
    ('Log(assets)²', theta_final[4], 'Diminishing size effect'),
    ('NPL ratio', theta_final[5], 'NPL effect (residual after MC)'),
    ('ROA', theta_final[6], 'Profitability reduces NMC'),
    ('Real estate', theta_final[7], 'Real estate effect (residual)'),
    ('House index', theta_final[8], 'Political index effect'),
    ('Senate index', theta_final[9], 'Political index effect'),
]

nmc_table = [[name, f"{est:.2f}", interp] for name, est, interp in nmc_data]

print(tabulate(nmc_table,
               headers=['Variable', 'Estimate', 'Interpretation'],
               tablefmt='simple'))

In [ ]:
# Final visualization
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Plot 1: Predicted P(close) vs equity
axes[0].scatter(equity_a_all[:, 0], Eprob_now_adj, alpha=0.1, s=1)
axes[0].set_xlabel('Equity / Assets')
axes[0].set_ylabel('P(Close)')
axes[0].set_title('Closure Probability vs Capital Ratio')
axes[0].axvline(x=0.08, color='red', linestyle='--', label='Regulatory min (8%)')
axes[0].legend()

# Plot 2: Predicted P(close) vs NPL
axes[1].scatter(npf_a_all[:, 0], Eprob_now_adj, alpha=0.1, s=1)
axes[1].set_xlabel('Non-Performing Loans / Assets')
axes[1].set_ylabel('P(Close)')
axes[1].set_title('Closure Probability vs Asset Quality')

plt.tight_layout()
plt.show()